### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

❗️ **Modify entries under _1) User-defined parameters_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
apim_sku    = APIM_SKU.STANDARDV2
deployment  = INFRASTRUCTURE.AFD_APIM_PE
use_ACA     = True

# 2) Service-defined parameters (please do not change these unless you know what you're doing)
rg_name             = utils.get_infra_rg_name(deployment, index)
rg_tags             = utils.build_infrastructure_tags(deployment)
apim_network_mode   = APIMNetworkMode.EXTERNAL_VNET

# 3) Define the APIs and their operations and policies

# Policies
hello_world_policy_xml  = utils.read_policy_xml(HELLO_WORLD_XML_POLICY_PATH)

# Hello World (Root)
api_hwroot_get  = GET_APIOperation('This is a GET for API 1', hello_world_policy_xml)
api_hwroot      = API('hello-world', 'Hello World', '', 'This is the root API for Hello World', operations = [api_hwroot_get])

apis: List[API] = [api_hwroot]

# If Container Apps is enabled, create the ACA APIs in APIM
if use_ACA:
    utils.print_info('ACA APIs will be created.')

    aca_backend_1_policy_xml    = utils.read_policy_xml(BACKEND_XML_POLICY_PATH).format(backend_id = 'aca-backend-1')
    aca_backend_2_policy_xml    = utils.read_policy_xml(BACKEND_XML_POLICY_PATH).format(backend_id = 'aca-backend-2')
    aca_backend_pool_policy_xml = utils.read_policy_xml(BACKEND_XML_POLICY_PATH).format(backend_id = 'aca-backend-pool')

    # Hello World (ACA Backend 1)
    api_hwaca_1_get = GET_APIOperation('This is a GET for Hello World on ACA Backend 1')
    api_hwaca_1 = API('hello-world-aca-1', 'Hello World (ACA 1)', '/aca-1', 'This is the ACA API for Backend 1', policyXml = aca_backend_1_policy_xml, operations = [api_hwaca_1_get])

    # Hello World (ACA Backend 2)
    api_hwaca_2_get = GET_APIOperation('This is a GET for Hello World on ACA Backend 2')
    api_hwaca_2 = API('hello-world-aca-2', 'Hello World (ACA 2)', '/aca-2', 'This is the ACA API for Backend 2', policyXml = aca_backend_2_policy_xml, operations = [api_hwaca_2_get])

    # Hello World (ACA Backend Pool)
    api_hwaca_pool_get = GET_APIOperation('This is a GET for Hello World on ACA Backend Pool')
    api_hwaca_pool = API('hello-world-aca-pool', 'Hello World (ACA Pool)', '/aca-pool', 'This is the ACA API for Backend Pool', policyXml = aca_backend_pool_policy_xml, operations = [api_hwaca_pool_get])

    # Add ACA APIs to the existing apis array
    apis += [api_hwaca_1, api_hwaca_2, api_hwaca_pool]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils
from apimtypes import *

# 1) Define the Bicep parameters with serialized APIs and networking mode
bicep_parameters = {
    'apimSku'           : {'value': apim_sku.value},
    'apis'              : {'value': [api.to_dict() for api in apis]},
    'apimPublicAccess'  : {'value': apim_network_mode in [APIMNetworkMode.PUBLIC, APIMNetworkMode.EXTERNAL_VNET]},
    'useACA'            : {'value': use_ACA}
}

# 2) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters, rg_tags = rg_tags)

# 3) Print a deployment summary, if successful; otherwise, exit with an error
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_service_id     = output.get('apimServiceId', 'APIM Service Id')
    apim_gateway_url    = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    afd_endpoint_url    = output.get('fdeSecureUrl', 'Front Door Endpoint URL')

utils.print_ok('Deployment completed')


### 🔗 3. Approve Front Door private link connection to APIM

In the deployed Bicep template, Azure Front Door will establish a private link connection to the API Management service. This connection should be approved. Run the following command to approve the connection.

In [ ]:
import utils

# Get all pending private endpoint connections as JSON
output = utils.run(f"az network private-endpoint-connection list --id {apim_service_id} --query \"[?contains(properties.privateLinkServiceConnectionState.status, 'Pending')]\" -o json")

# Handle both a single object and a list of objects
pending_connections = output.json_data if output.success and output.is_json else []

if isinstance(pending_connections, dict):
    pending_connections = [pending_connections]

total = len(pending_connections)
utils.print_info(f"Found {total} pending private link service connection(s).")

if total > 0:
    for i, conn in enumerate(pending_connections, 1):
        conn_id = conn.get('id')
        conn_name = conn.get('name', '<unknown>')
        utils.print_info(f"{i}/{total}: {conn_name}", True)

        approve_result = utils.run(
            f"az network private-endpoint-connection approve --id {conn_id} --description 'Approved'",
            f"Private Link Connection approved: {conn_name}",
            f"Failed to approve Private Link Connection: {conn_name}"
        )

    utils.print_ok('Private link approvals completed')
else:
    utils.print_info('No pending private link service connection was found. There is nothing to approve.')

### ✅ 4. Verify API Request Success via API Management

As we have not yet disabled public access to APIM, this request should succeed with a **200**.

In [ ]:
import utils
from apimrequests import ApimRequests

reqs = ApimRequests(apim_gateway_url)

utils.print_message('Calling Hello World (Root) API via API Management Gateway URL. Expect 200 (if run before disabling API Management public network access).')
output = reqs.singleGet('/')

utils.print_ok('API request via API Management completed')

### 🔒 5. Disabling API Management public network access

The initial `APIM` service deployment above cannot disable public network access. It must be disabled subsequently below.

In [ ]:
import utils
from apimtypes import *

# 1) Update the Bicep parameters to disable public access to APIM (we only want private endpoint ingress)
bicep_parameters['apimPublicAccess']['value'] = False

# 2) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters)

# 3) Print a single, clear deployment summary if successful
if not output.success:
    raise SystemExit('Deployment failed')
    
if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    afd_endpoint_url = output.get('fdeSecureUrl', 'Front Door Endpoint URL')

utils.print_ok('Deployment completed')


### ✅ 6. Verify API Request Success via Azure Front Door & Failure with API Management

At this time only requests through Front Door should be successful and return a **200**. Requests to APIM that worked previously should result in a **403**.

In [ ]:
import utils
from apimrequests import ApimRequests

reqsApim = ApimRequests(apim_gateway_url)
reqsAfd  = ApimRequests(afd_endpoint_url)

# 1) Unsuccessful call to APIM Gateway URL (should fail with 403 Forbidden)
reqsApim.singleGet('/', msg = '1) Calling Hello World (Root) API via API Management Gateway URL. Expect 403 as APIM public access is disabled now.')

# 2) Successful call to Front Door (200)
reqsAfd.singleGet('/', msg = '2) Calling Hello World (Root) API via Azure Front Door. Expect 200.')

# 3) Successful calls to Front Door -> APIM -> ACA (200)
if use_ACA:
    reqsAfd.singleGet('/aca-1', msg = '3) Calling Hello World (ACA 1) API via Azure Front Door. Expect 200.')
    reqsAfd.singleGet('/aca-2', msg = '4) Calling Hello World (ACA 2) API via Azure Front Door. Expect 200.')
    reqsAfd.singleGet('/aca-pool', msg = '5) Calling Hello World (ACA Pool) API via Azure Front Door. Expect 200.')
else:
    utils.print_message('ACA APIs were not created. Skipping ACA API calls.', blank_above = True)

utils.print_ok('All done!')

### 🗑️ Clean up resources

When you're finished experimenting, it's advisable to remove all associated resources from Azure to avoid unnecessary cost.
Use the [clean-up notebook](clean-up.ipynb) for that.